# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Import cities csv as data frame
cities_file_path=os.path.join("output_data","cities.csv")
weather_df=pd.read_csv(cities_file_path)
weather_0_df=weather_df.loc[:,["City","Latitude","Longitude","Max Temp","Humidity","Cloudiness","Wind Speed","Country","Date"]]
weather_0_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Letterkenny,54.9500,-7.7333,274.06,88,100,4.16,IE,1612052820
1,Kapaa,22.0752,-159.3190,299.15,65,75,6.69,US,1612052817
2,Seymchan,62.8833,152.4333,239.40,80,100,4.77,RU,1612052821
3,Phan Thiet,10.9333,108.1000,295.04,72,100,2.17,VN,1612052821
4,Tuatapere,-46.1333,167.6833,291.85,47,93,0.59,NZ,1612052821


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)


# Store latitude and longitude in locations
locations = weather_0_df[["Latitude", "Longitude"]]
# coordinates=zip(weather_df['Latitude'],weather_df['Longtitude'])

#Store Weights for Heat Map
humidity=weather_0_df["Humidity"].astype(float)

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
#Plot Heat Map
fig=gmaps.figure(layout=figure_layout)

#Create Heatmap Layer
heat_layer= gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=100,point_radius=5)


fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Ideal Weather Conditions
    # 75 < Max Temperature < 80 Farenheit, which is ~297 to 300 K
    # Wind Speed < 3
    # Humidity < 90
    # Cloudiness < 30

#Filter Original Data Frame to Ideal Weather Conditions
weather_filter=(weather_0_df["Max Temp"]>297)&(weather_0_df["Max Temp"]<300) &(weather_0_df["Wind Speed"]<2)&(weather_0_df["Humidity"]<90)&(weather_0_df["Cloudiness"]<30)

#New Hotel Search Data Frame
hotel_df=weather_0_df.loc[weather_filter,:]
hotel_df
# len(hotel_df)

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
140,Mahajanga,-15.7167,46.3167,297.15,88,20,1.54,MG,1612052844
204,Yenagoa,4.9247,6.2642,299.03,86,0,1.66,NG,1612052856
273,Malpe,13.3500,74.7167,297.15,83,1,1.47,IN,1612052870
311,Irámuco,19.9500,-100.9167,299.15,19,1,1.03,MX,1612052876
313,Warri,5.5167,5.7500,299.20,84,0,0.83,NG,1612052876
423,Saint-Louis,16.3333,-15.0000,298.15,33,0,1.03,SN,1612052896
467,Mangaratiba,-22.9597,-44.0406,299.15,78,0,1.54,BR,1612052905
512,Grand Gaube,-20.0064,57.6608,297.59,86,6,1.34,MU,1612052848


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#Starter URL: https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set the parameters used in the API call
lat_long=''
distance=5000
target_type='lodging'

query_parameters={
    'key': g_key,
    'location':lat_long,
    'radius':distance,
    'type':target_type
}

#Using iterrows, iterate through Hotel Data Frame, use Latitude 
#and Longitude to find nearest hotel and store it in Data Frame 
#under Hotel Name column

# for index, each_row in hotel_df.head(5).iterrows():
for index, each_row in hotel_df.iterrows():
    
    try:
        #Get Latitude, Lontitude List
        query_parameters['location']=f"{each_row['Latitude']}, {each_row['Longitude']}"
    #     print(lat_long)

        # Compile Query URL and make API request
        response=requests.get(base_url,params=query_parameters)
        data=response.json()
#         pprint(data)

        #Get Hotel name and add it to the Hotel Data Frame
        hotel_df.loc[index,"Hotel Name"]=data['results'][0]['name']
    
    except:
        print(f"Unable to locate a hotel in {each_row['City']} within {distance} meters.")
#         print(f"Unable to locate a hotel in {each_row['City']} within {distance} meters of {query_parameters['location']} coordinates.")

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Unable to locate a hotel in Saint-Louis within 5000 meters.


In [6]:
#Drop any row where there is an NaN, find number of cities have hotel within 500 meter
hotel_1_df=hotel_df.dropna(axis=0,how='any')
len(hotel_1_df)

7

In [7]:
#Number of cities in original hotel data frame, before dropping NaN
len(hotel_df)

8

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_1_df.iterrows()]
locations = hotel_1_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
# fig=gmaps.figure()

markers=gmaps.marker_layer(locations,info_box_content =hotel_info)
# markers=gmaps.marker_layer(locations,hover_text=hotel_info)

fig.add_layer(heat_layer)

fig.add_layer(markers)

# Display figure

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…